# 🚀 Token Rate Limiting Lab with Azure API Management

This lab demonstrates implementing **token-based rate limiting** using Azure API Management's `llm-token-limit` policy with Azure OpenAI.

## 🎯 What You'll Learn
- Deploy Azure APIM (BasicV2 SKU) as an AI Gateway
- Deploy Azure OpenAI with GPT-4o-mini model
- Configure token-based rate limiting (500 TPM)
- Monitor and test rate limiting behavior

## 📚 Official Microsoft Documentation & Learning Resources

| Resource | Description |
|----------|-------------|
| [Azure API Management GenAI Gateway](https://learn.microsoft.com/azure/api-management/genai-gateway-capabilities) | Overview of AI gateway capabilities in APIM |
| [llm-token-limit Policy](https://learn.microsoft.com/azure/api-management/llm-token-limit-policy) | Official docs for token-based rate limiting policy |
| [Azure OpenAI Service](https://learn.microsoft.com/azure/ai-services/openai/overview) | Azure OpenAI documentation |
| [APIM Pricing Tiers](https://learn.microsoft.com/azure/api-management/api-management-features) | Understanding APIM SKUs (BasicV2 used here) |
| [MS Learn: Implement API Management](https://learn.microsoft.com/training/modules/explore-api-management/) | Free training module on APIM |
| [MS Learn: Azure OpenAI](https://learn.microsoft.com/training/paths/develop-ai-solutions-azure-openai/) | Learning path for Azure OpenAI |

---

## 🏗️ Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────────────────┐
│                              Azure Subscription                                  │
│  ┌───────────────────────────────────────────────────────────────────────────┐  │
│  │                    Resource Group: lab-token-rate-limiting                │  │
│  │                                                                           │  │
│  │   ┌─────────────┐         ┌──────────────────────┐         ┌──────────┐  │  │
│  │   │   Client    │         │   Azure API Mgmt     │         │  Azure   │  │  │
│  │   │ Application │ ──────► │   (BasicV2 SKU)      │ ──────► │  OpenAI  │  │  │
│  │   │             │ HTTP(S) │                      │  HTTP   │          │  │  │
│  │   └─────────────┘         │  ┌────────────────┐  │         │ GPT-4o-  │  │  │
│  │                           │  │ llm-token-limit│  │         │  mini    │  │  │
│  │                           │  │    Policy      │  │         │          │  │  │
│  │                           │  │  (500 TPM)     │  │         └──────────┘  │  │
│  │                           │  └────────────────┘  │                       │  │
│  │                           │                      │                       │  │
│  │                           │  Features:           │                       │  │
│  │                           │  • Token counting    │                       │  │
│  │                           │  • Rate enforcement  │                       │  │
│  │                           │  • 429 responses     │                       │  │
│  │                           └──────────────────────┘                       │  │
│  │                                                                           │  │
│  └───────────────────────────────────────────────────────────────────────────┘  │
└─────────────────────────────────────────────────────────────────────────────────┘
```

### How Token Rate Limiting Works

1. **Client Request**: Application sends a chat completion request to APIM
2. **Token Estimation**: APIM estimates tokens in the request using a built-in tokenizer
3. **Rate Check**: The `llm-token-limit` policy checks against the 500 TPM quota
4. **Response Counting**: Response tokens are counted and added to the consumption total
5. **Enforcement**: If limit exceeded, APIM returns `429 Too Many Requests`

> 📖 **Learn more**: [How token limiting works](https://learn.microsoft.com/azure/api-management/llm-token-limit-policy#how-token-limiting-works)

---

## ✅ Prerequisites

| Requirement | Description | Documentation |
|-------------|-------------|---------------|
| Azure Subscription | With Contributor access to create resources | [Create free account](https://azure.microsoft.com/free/) |
| Azure CLI | Version 2.50+ installed and configured | [Install Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) |
| Python 3.8+ | With pip for package management | [Python downloads](https://www.python.org/downloads/) |
| Azure OpenAI Access | Approved access to Azure OpenAI Service | [Request access](https://aka.ms/oai/access) |

> 💡 **Tip**: Run `az version` to verify your Azure CLI installation

## Step 1: Login to Azure

Authenticate with Azure using the interactive login flow. This will open a browser window for authentication.

> 📖 **Reference**: [Sign in with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli)

In [ ]:
!az login

## Step 2: Set Configuration Variables

Configure the deployment parameters for your lab environment. A unique suffix is generated to ensure globally unique resource names.

| Variable | Description | Value |
|----------|-------------|-------|
| `RESOURCE_GROUP` | Azure resource group name | `lab-token-rate-limiting` |
| `LOCATION` | Azure region | `swedencentral` |
| `APIM_NAME` | API Management instance name | Auto-generated |
| `OPENAI_NAME` | Azure OpenAI resource name | Auto-generated |
| `MODEL_NAME` | OpenAI model deployment | `gpt-4o-mini` |
| `TPM_LIMIT` | Tokens Per Minute limit | `500` |

> 📖 **Reference**: [Azure regions with OpenAI availability](https://learn.microsoft.com/azure/ai-services/openai/concepts/models#model-summary-table-and-region-availability)

In [ ]:
import os
import random
import string

# Generate unique suffix
suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))

# Configuration
RESOURCE_GROUP = "lab-token-rate-limiting"
LOCATION = "swedencentral"
APIM_NAME = f"apim-tokenratelimit-{suffix}"
OPENAI_NAME = f"openai-tokenratelimit-{suffix}"
MODEL_NAME = "gpt-4o-mini"
TPM_LIMIT = 500

print(f"APIM Name: {APIM_NAME}")
print(f"OpenAI Name: {OPENAI_NAME}")

## Step 3: Create Resource Group

Create an Azure resource group to contain all lab resources. Resource groups provide a logical container for managing and organizing related resources.

> 📖 **Reference**: [Manage Azure resource groups](https://learn.microsoft.com/azure/azure-resource-manager/management/manage-resource-groups-cli)

In [ ]:
!az group create --name {RESOURCE_GROUP} --location {LOCATION}

## Step 4: Deploy Infrastructure

Deploy the complete infrastructure using Bicep templates:

| Resource | SKU/Tier | Purpose |
|----------|----------|---------|
| Azure API Management | BasicV2 | AI Gateway with GenAI policies support |
| Azure OpenAI | Standard | LLM hosting (GPT-4o-mini) |

**⏱️ Deployment Time**: ~5-10 minutes (APIM BasicV2 deploys faster than legacy tiers)

> 📖 **References**: 
> - [Deploy Bicep files with Azure CLI](https://learn.microsoft.com/azure/azure-resource-manager/bicep/deploy-cli)
> - [APIM v2 tiers overview](https://learn.microsoft.com/azure/api-management/v2-service-tiers-overview)

In [ ]:
!az deployment group create --resource-group {RESOURCE_GROUP} --template-file main.bicep --parameters apimName={APIM_NAME} openAiName={OPENAI_NAME} location={LOCATION}

## Step 5: Get APIM Subscription Key

Retrieve the API Management gateway URL and subscription key for authentication. APIM uses subscription keys to control access to APIs.

> 📖 **Reference**: [Subscriptions in Azure API Management](https://learn.microsoft.com/azure/api-management/api-management-subscriptions)

In [ ]:
import subprocess, json

result = subprocess.run(["az", "apim", "show", "--name", APIM_NAME, "--resource-group", RESOURCE_GROUP, "--query", "gatewayUrl", "-o", "tsv"], capture_output=True, text=True)
GATEWAY_URL = result.stdout.strip()

result = subprocess.run(["az", "apim", "subscription", "keys", "list", "--resource-group", RESOURCE_GROUP, "--service-name", APIM_NAME, "--subscription-id", "aoai-subscription", "-o", "json"], capture_output=True, text=True)
API_KEY = json.loads(result.stdout).get("primaryKey", "")

print(f"Gateway URL: {GATEWAY_URL}")
print(f"API Key: {API_KEY}")

## Step 6: Test Rate Limiting

Send multiple requests to observe the `llm-token-limit` policy in action. With a 500 TPM limit, you should see rate limiting kick in after a few requests.

### Expected Behavior

| Response Code | Meaning | Action |
|---------------|---------|--------|
| `200 OK` | Request successful | Tokens counted against quota |
| `429 Too Many Requests` | Rate limit exceeded | Wait for quota reset |

> 📖 **Reference**: [llm-token-limit policy error handling](https://learn.microsoft.com/azure/api-management/llm-token-limit-policy#errors)

In [ ]:
import requests, time

endpoint = f"{GATEWAY_URL}/openai/deployments/{MODEL_NAME}/chat/completions?api-version=2024-02-01"
headers = {"api-key": API_KEY, "Content-Type": "application/json"}
payload = {"messages": [{"role": "user", "content": "Say hello in 50 words"}], "max_tokens": 100}

for i in range(10):
    r = requests.post(endpoint, headers=headers, json=payload, timeout=30)
    if r.status_code == 200:
        print(f"Request {i+1}: ✅ Success")
    elif r.status_code == 429:
        print(f"Request {i+1}: ⚠️ Rate limited")
    else:
        print(f"Request {i+1}: ❌ Error {r.status_code}")
    time.sleep(0.5)

## Step 7: Run Dashboard

Launch the Streamlit dashboard to visualize token consumption and rate limiting in real-time.

```bash
pip install -r dashboard/requirements.txt
cd dashboard && streamlit run app.py
```

Dashboard opens at http://localhost:8501

> 📖 **Reference**: [Streamlit documentation](https://docs.streamlit.io/)

---

## 📊 Additional Monitoring Options

For production scenarios, consider these Azure-native monitoring approaches:

| Tool | Use Case | Documentation |
|------|----------|---------------|
| Azure Monitor | Metrics and alerts for APIM | [Monitor API Management](https://learn.microsoft.com/azure/api-management/api-management-howto-use-azure-monitor) |
| Application Insights | End-to-end request tracing | [APIM with App Insights](https://learn.microsoft.com/azure/api-management/api-management-howto-app-insights) |
| Log Analytics | Query and analyze logs | [Diagnostic logs](https://learn.microsoft.com/azure/api-management/api-management-howto-use-azure-monitor#resource-logs) |

---

## 🧹 Cleanup

Delete all resources to avoid ongoing charges. The resource group deletion removes all contained resources.

> ⚠️ **Warning**: This action is irreversible. Uncomment the code below only when you're ready to delete all lab resources.

> 📖 **Reference**: [Delete resource groups](https://learn.microsoft.com/azure/azure-resource-manager/management/delete-resource-group)

In [ ]:
# Uncomment to delete resources
# !az group delete --name {RESOURCE_GROUP} --yes --no-wait

---

## 📖 Additional Resources

### Microsoft Learn Modules
- [Explore Azure API Management](https://learn.microsoft.com/training/modules/explore-api-management/) - Comprehensive APIM training
- [Develop AI solutions with Azure OpenAI](https://learn.microsoft.com/training/paths/develop-ai-solutions-azure-openai/) - Full learning path
- [Implement Azure OpenAI Service](https://learn.microsoft.com/training/modules/implement-azure-openai-service/) - Hands-on module

### GitHub Samples & References
- [Azure API Management GenAI Samples](https://github.com/Azure-Samples/AI-Gateway) - Official AI Gateway samples
- [Azure OpenAI Samples](https://github.com/Azure-Samples/openai) - OpenAI code samples

### Related Policies
| Policy | Purpose | Documentation |
|--------|---------|---------------|
| `llm-token-limit` | Token-based rate limiting | [Docs](https://learn.microsoft.com/azure/api-management/llm-token-limit-policy) |
| `llm-emit-token-metric` | Emit token usage metrics | [Docs](https://learn.microsoft.com/azure/api-management/llm-emit-token-metric-policy) |
| `llm-semantic-cache-store` | Semantic caching for LLMs | [Docs](https://learn.microsoft.com/azure/api-management/llm-semantic-cache-store-policy) |
| `azure-openai-emit-token-metric` | Azure OpenAI token metrics | [Docs](https://learn.microsoft.com/azure/api-management/azure-openai-emit-token-metric-policy) |

---

**Lab Version**: 1.0 | **Last Updated**: February 2026 | **Author**: Azure AI Gateway Team